# Modeling and solving search problems

Consider the following river crossing puzzle:

## Jealous Husbands Problem with an Island

There are n couples: $(𝐻_1,𝑊_1)$, $(𝐻_2,𝑊_2)$, …, $(𝐻_𝑛,𝑊_𝑛)$  where $𝐻_𝑖$ is husband $𝑖$ and $𝑊_𝑖$ is wife $𝑖$.

They need to cross from the left bank of a river to the right bank.

There is a boat that can carry at most two people at a time.

Important: There is an island midway between the left and right banks. The boat can stop at the island.

Rules:

1. Jealousy constraint: No wife may ever be in the presence of another husband (on any shore or the island or in the boat) unless her own husband is also present. Otherwise, fights (and failures) occur.

2. The boat needs at least one person to operate (no remote-controlled trips).

3. The boat can travel between:

            Left bank ↔ Island

            Island ↔ Right bank

4. All people, including the boat, start at the left bank.

Goal: get everyone (husbands and wives) safely to the right bank, obeying all the rules.


## Tasks:

1. Put the problem in a form of a search tree: root, nodes, solution and rules for constructing the childrens.
2. Solve it usinh a classic DFS
3. Solve it using a classic Back-tracking.
4. Solve it using the A* method.  

In [51]:
# SETUP

from itertools import combinations, chain, count
from heapq import heappush, heappop
from collections import deque

# Number of couples
N = 3
people = [f'H{i+1}' for i in range(N)] + [f'W{i+1}' for i in range(N)]

In [53]:
# State represented as tuple of (left, island, right, boat_pos)
class State:
    def __init__(self, left, island, right, boat):
        self.left = frozenset(left)
        self.island = frozenset(island)
        self.right = frozenset(right)
        self.boat = boat  # 'left', 'island', or 'right'

    def is_goal(self):
        return len(self.right) == 2 * N

    def __hash__(self):
        return hash((self.left, self.island, self.right, self.boat))

    def __eq__(self, other):
        return (self.left, self.island, self.right, self.boat) == (other.left, other.island, other.right, other.boat)

    def __str__(self):
        return f"L:{sorted(self.left)} | I:{sorted(self.island)} | R:{sorted(self.right)} | Boat:{self.boat}"

# Initial state
initial_state = State(set(people), set(), set(), 'left')

In [55]:
# Helper: Jealousy check
def is_valid(group):
    for w in [p for p in group if p.startswith('W')]:
        h = 'H' + w[1:]
        if h not in group:
            for h_other in [p for p in group if p.startswith('H')]:
                if h_other != h:
                    return False
    return True

# Generator for all valid children from current state
def generate_children(state):
    locs = {
        'left': set(state.left),
        'island': set(state.island),
        'right': set(state.right)
    }

    transitions = {
        'left': 'island',
        'island': 'right',
        'right': 'island',
        'island-back': 'left'
    }

    results = []
    for direction in ['left', 'island', 'right']:
        if state.boat != direction:
            continue

        to = transitions[direction] if direction != 'island' else transitions['island']  # island -> right
        from_group = locs[direction]
        to_group = locs[to]

        for people_in_boat in chain(combinations(from_group, 1), combinations(from_group, 2)):
            new_from = from_group.difference(people_in_boat)
            new_to = to_group.union(people_in_boat)

            new_locs = dict(locs)
            new_locs[direction] = new_from
            new_locs[to] = new_to

            if all(is_valid(new_locs[loc]) for loc in ['left', 'island', 'right']) and is_valid(people_in_boat):
                new_boat = to
                new_state = State(new_locs['left'], new_locs['island'], new_locs['right'], new_boat)
                results.append(new_state)

        if direction == 'island':
            to = transitions['island-back']
            to_group = locs[to]
            for people_in_boat in chain(combinations(from_group, 1), combinations(from_group, 2)):
                new_from = from_group.difference(people_in_boat)
                new_to = to_group.union(people_in_boat)
                new_locs = dict(locs)
                new_locs['island'] = new_from
                new_locs[to] = new_to
                if all(is_valid(new_locs[loc]) for loc in ['left', 'island', 'right']) and is_valid(people_in_boat):
                    new_state = State(new_locs['left'], new_locs['island'], new_locs['right'], to)
                    results.append(new_state)

    return results

In [57]:
# DFS implementation
def dfs(state, visited=None):
    if visited is None:
        visited = set()
    if state.is_goal():
        return [state]
    visited.add(state)
    for child in generate_children(state):
        if child not in visited:
            result = dfs(child, visited)
            if result:
                return [state] + result
    return None

# Run DFS
dfs_result = dfs(initial_state)
dfs_result_str = [str(s) for s in dfs_result] if dfs_result else []

# Show results
dfs_result_str

["L:['H1', 'H2', 'H3', 'W1', 'W2', 'W3'] | I:[] | R:[] | Boat:left",
 "L:['H1', 'H2', 'W1', 'W2'] | I:['H3', 'W3'] | R:[] | Boat:island",
 "L:['H1', 'H2', 'W1', 'W2'] | I:[] | R:['H3', 'W3'] | Boat:right",
 "L:['H1', 'H2', 'W1', 'W2'] | I:['H3'] | R:['W3'] | Boat:island",
 "L:['H1', 'H2', 'H3', 'W1', 'W2'] | I:[] | R:['W3'] | Boat:left",
 "L:['H1', 'H2', 'H3', 'W2'] | I:['W1'] | R:['W3'] | Boat:island",
 "L:['H1', 'H2', 'H3', 'W2'] | I:[] | R:['W1', 'W3'] | Boat:right",
 "L:['H1', 'H2', 'H3', 'W2'] | I:['W3'] | R:['W1'] | Boat:island",
 "L:['H1', 'H2', 'H3', 'W2', 'W3'] | I:[] | R:['W1'] | Boat:left",
 "L:['H2', 'H3', 'W2', 'W3'] | I:['H1'] | R:['W1'] | Boat:island",
 "L:['H2', 'H3', 'W2', 'W3'] | I:[] | R:['H1', 'W1'] | Boat:right",
 "L:['H2', 'H3', 'W2', 'W3'] | I:['H1', 'W1'] | R:[] | Boat:island",
 "L:['H1', 'H2', 'H3', 'W2', 'W3'] | I:['W1'] | R:[] | Boat:left",
 "L:['H1', 'H2', 'H3', 'W2'] | I:['W1', 'W3'] | R:[] | Boat:island",
 "L:['H1', 'H2', 'H3', 'W1', 'W2'] | I:['W3'] | R:[

In [59]:
# Backtracking implementation
def backtrack(state, path=None, visited=None):
    if visited is None:
        visited = set()
    if path is None:
        path = []

    if state.is_goal():
        return path + [state]
    visited.add(state)
    for child in generate_children(state):
        if child not in visited:
            result = backtrack(child, path + [state], visited)
            if result:
                return result
    return None

# Run Backtracking
backtrack_result = backtrack(initial_state)
backtrack_result_str = [str(s) for s in backtrack_result] if backtrack_result else []

# Show results
backtrack_result_str

["L:['H1', 'H2', 'H3', 'W1', 'W2', 'W3'] | I:[] | R:[] | Boat:left",
 "L:['H1', 'H2', 'W1', 'W2'] | I:['H3', 'W3'] | R:[] | Boat:island",
 "L:['H1', 'H2', 'W1', 'W2'] | I:[] | R:['H3', 'W3'] | Boat:right",
 "L:['H1', 'H2', 'W1', 'W2'] | I:['H3'] | R:['W3'] | Boat:island",
 "L:['H1', 'H2', 'H3', 'W1', 'W2'] | I:[] | R:['W3'] | Boat:left",
 "L:['H1', 'H2', 'H3', 'W2'] | I:['W1'] | R:['W3'] | Boat:island",
 "L:['H1', 'H2', 'H3', 'W2'] | I:[] | R:['W1', 'W3'] | Boat:right",
 "L:['H1', 'H2', 'H3', 'W2'] | I:['W3'] | R:['W1'] | Boat:island",
 "L:['H1', 'H2', 'H3', 'W2', 'W3'] | I:[] | R:['W1'] | Boat:left",
 "L:['H2', 'H3', 'W2', 'W3'] | I:['H1'] | R:['W1'] | Boat:island",
 "L:['H2', 'H3', 'W2', 'W3'] | I:[] | R:['H1', 'W1'] | Boat:right",
 "L:['H2', 'H3', 'W2', 'W3'] | I:['H1', 'W1'] | R:[] | Boat:island",
 "L:['H1', 'H2', 'H3', 'W2', 'W3'] | I:['W1'] | R:[] | Boat:left",
 "L:['H1', 'H2', 'H3', 'W2'] | I:['W1', 'W3'] | R:[] | Boat:island",
 "L:['H1', 'H2', 'H3', 'W1', 'W2'] | I:['W3'] | R:[

In [61]:
# Simple Heuristic for A* search
def heuristic(state):
    return len(state.left)

# A* implementation
def a_star(start_state):
    frontier = []
    counter = count()
    heappush(frontier, (heuristic(start_state), 0, next(counter), start_state, []))
    visited = set()
    while frontier:
        f, g, _, current, path = heappop(frontier)
        if current in visited:
            continue
        visited.add(current)
        if current.is_goal():
            return path + [current]
        for child in generate_children(current):
            if child not in visited:
                heappush(frontier, (g + 1 + heuristic(child), g + 1, next(counter), child, path + [current]))
    return None
    
# Run A*
astar_result = a_star(initial_state)
astar_result_str = [str(s) for s in astar_result] if astar_result else []

# Show results
astar_result_str

["L:['H1', 'H2', 'H3', 'W1', 'W2', 'W3'] | I:[] | R:[] | Boat:left",
 "L:['H1', 'H2', 'W1', 'W2'] | I:['H3', 'W3'] | R:[] | Boat:island",
 "L:['H1', 'H2', 'H3', 'W1', 'W2'] | I:['W3'] | R:[] | Boat:left",
 "L:['H1', 'H2', 'H3'] | I:['W1', 'W2', 'W3'] | R:[] | Boat:island",
 "L:['H1', 'H2', 'H3', 'W2'] | I:['W1', 'W3'] | R:[] | Boat:left",
 "L:['H2', 'W2'] | I:['H1', 'H3', 'W1', 'W3'] | R:[] | Boat:island",
 "L:['H2', 'W2'] | I:['H3', 'W3'] | R:['H1', 'W1'] | Boat:right",
 "L:['H2', 'W2'] | I:['H1', 'H3', 'W3'] | R:['W1'] | Boat:island",
 "L:['H1', 'H2', 'W2'] | I:['H3', 'W3'] | R:['W1'] | Boat:left",
 "L:['W2'] | I:['H1', 'H2', 'H3', 'W3'] | R:['W1'] | Boat:island",
 "L:['W2'] | I:['H3', 'W3'] | R:['H1', 'H2', 'W1'] | Boat:right",
 "L:['W2'] | I:['H2', 'H3', 'W3'] | R:['H1', 'W1'] | Boat:island",
 "L:['H2', 'W2'] | I:['H3', 'W3'] | R:['H1', 'W1'] | Boat:left",
 "L:[] | I:['H2', 'H3', 'W2', 'W3'] | R:['H1', 'W1'] | Boat:island",
 "L:[] | I:['W2', 'W3'] | R:['H1', 'H2', 'H3', 'W1'] | Boa